In [ ]:
!pip install dmba

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import dmba
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from dmba import plotDecisionTree, classificationSummary
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical

In [ ]:
nba_df=pd.read_excel("NBA_2024_Shots.xlsx")
nba_df.head(5)
nba_df["SHOT_DISTANCE"].head(5)


,SHOT_DISTANCE
0,12
1,25
2,3
3,1
4,1


In [ ]:
nba_df["ZONE_RANGE"].unique()

array(['8-16 ft.', '24+ ft.', 'Less Than 8 ft.', '16-24 ft.',
       'Back Court Shot'], dtype=object)

In [ ]:
nba_df=nba_df.dropna(subset=['POSITION_GROUP','POSITION'])
New_number_of_records=len(nba_df)
print("New number of records:",New_number_of_records)


New number of records: 217437


In [ ]:
POSITION_GROUP_DUMMY=pd.get_dummies(nba_df['POSITION_GROUP'],drop_first=True).astype(int)
POSITION_DUMMY=pd.get_dummies(nba_df['POSITION'],drop_first=True).astype(int)
HOME_TEAM_DUMMY=pd.get_dummies(nba_df['HOME_TEAM'],drop_first=True).astype(int)
AWAY_TEAM_DUMMY=pd.get_dummies(nba_df['AWAY_TEAM'],drop_first=True).astype(int)
EVENT_TYPE_DUMMY=pd.get_dummies(nba_df['EVENT_TYPE'],drop_first=True).astype(int)
ACTION_TYPE_DUMMY=pd.get_dummies(nba_df['ACTION_TYPE'],drop_first=True).astype(int)
SHOT_TYPE_DUMMY=pd.get_dummies(nba_df['SHOT_TYPE'],drop_first=True).astype(int)
BASIC_ZONE_DUMMY=pd.get_dummies(nba_df['BASIC_ZONE'],drop_first=True).astype(int)
ZONE_NAME_DUMMY=pd.get_dummies(nba_df['ZONE_NAME'],drop_first=True).astype(int)
ZONE_ABB_DUMMY=pd.get_dummies(nba_df['ZONE_ABB'],drop_first=True).astype(int)
ZONE_RANGE_DUMMY=pd.get_dummies(nba_df['ZONE_RANGE'],drop_first=True).astype(int)
nba_df=pd.concat([nba_df,POSITION_GROUP_DUMMY, POSITION_DUMMY,HOME_TEAM_DUMMY,AWAY_TEAM_DUMMY,EVENT_TYPE_DUMMY,ACTION_TYPE_DUMMY,SHOT_TYPE_DUMMY,BASIC_ZONE_DUMMY,ZONE_NAME_DUMMY,ZONE_ABB_DUMMY,ZONE_RANGE_DUMMY],axis=1)
nba_df["SHOT_MADE"]=nba_df["SHOT_MADE"].astype(int)
nba_df["SHOT_MADE"].head(5)
X = nba_df.drop(['SHOT_MADE','SEASON_1','SEASON_2',"GAME_DATE","GAME_ID", "TEAM_NAME","TEAM_ID","PLAYER_ID","PLAYER_NAME","POSITION_GROUP","POSITION","HOME_TEAM","AWAY_TEAM","EVENT_TYPE","ACTION_TYPE","SHOT_TYPE","BASIC_ZONE","ZONE_NAME","ZONE_ABB","ZONE_RANGE"],axis=1)
y=nba_df["SHOT_MADE"]
X_train, X_test, y_train,y_test=train_test_split(X,y, test_size=0.3, random_state=42)

In [ ]:
nn1=MLPClassifier(hidden_layer_sizes=(5,2), activation= "relu", max_iter = 3000)
nn1.fit(X_train,y_train)

MLPClassifier(hidden_layer_sizes=(5, 2), max_iter=3000)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score_nn1=accuracy_score(y_test,nn1.predict(X_test))
print("Accuracy Score:",accuracy_score_nn1)
classification_report(y_test,nn1.predict(X_test))

Accuracy Score: 1.0


'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     34237\n           1       1.00      1.00      1.00     30995\n\n    accuracy                           1.00     65232\n   macro avg       1.00      1.00      1.00     65232\nweighted avg       1.00      1.00      1.00     65232\n'

In [ ]:
X1=X.drop(['Missed Shot'],axis=1)
y1=y
X1_train, X1_test, y1_train,y1_test=train_test_split(X1,y1, test_size=0.3, random_state=42)


In [ ]:
nn1=MLPClassifier(hidden_layer_sizes=(3,3), activation= "relu", max_iter = 3000)
nn1.fit(X1_train,y1_train)
accuracy_score_2=accuracy_score(y1_test,nn1.predict(X1_test))
print("Accuracy Score:",accuracy_score_2)
classification_report(y1_test,nn1.predict(X1_test))


Accuracy Score: 0.6298595781211676


'              precision    recall  f1-score   support\n\n           0       0.60      0.86      0.71     34237\n           1       0.71      0.37      0.49     30995\n\n    accuracy                           0.63     65232\n   macro avg       0.66      0.62      0.60     65232\nweighted avg       0.65      0.63      0.61     65232\n'

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
import pandas as pd

# Train an MLPClassifier

# Calculate permutation importance
perm_importance = permutation_importance(nn1, X1_test, y1_test, scoring='accuracy')

# Extract feature importance
feature_importance = pd.DataFrame({
    'Feature': X1_train.columns,  # Feature indices
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)

print("Feature Importance (Permutation):")
print(feature_importance)









Feature Importance (Permutation):
                             Feature  Importance
2                      SHOT_DISTANCE    0.114168
237                        Jump Shot    0.017636
98                         Jump Shot    0.017636
1                              LOC_Y    0.010078
226               Driving Layup Shot    0.006819
..                               ...         ...
209                              PHX   -0.000141
192                              DAL   -0.000212
53                               DAL   -0.000212
84   Driving Floating Bank Jump Shot   -0.000239
223  Driving Floating Bank Jump Shot   -0.000239

[284 rows x 2 columns]


In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV

In [ ]:
#The hidden layer sizes and the number of nodes are randomly selected
param_grid = {
    "hidden_layer_sizes": [(1,2), (2,2), (2,3), (3,2), (3,3), (1,3,), (4,1), (4,4), (4,5), (5,2), (5,8), (8,8),(3,3,2)],
}
gridSearch = GridSearchCV(MLPClassifier(activation= "relu", max_iter = 3000, random_state=1, solver = "adam"),
                          param_grid, cv = 5, n_jobs = -1, return_train_score = True)
# -1 means using all processors
gridSearch.fit(X1_train, y1_train)
print('Best_score:', gridSearch.best_score_)
print("Best parameter:", gridSearch.best_params_)

#Based on the results, 2 hidden layesr with 1 node in the first layer and 2 nodes in the second layer is the best parameter for the neural network.


Best_score: 0.6326336191320916
Best parameter: {'hidden_layer_sizes': (3, 3)}
